# AIPI 590 - XAI | Explainable Techniques II
### Exploratory analysis of a dataset to determine the amount of correlation between features
### Mariam Khan

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mkhan13/XAI/blob/explainable_ml/explainable_techniques_II.ipynb)

In [5]:
import os
import pandas as pd

repo_name = "XAI"
branch_name = "explainable_ml"
git_path = "https://github.com/Mkhan13/XAI.git"

# If repo already exists, remove it first to avoid conflicts
if os.path.exists(repo_name):
    !rm -rf {repo_name}

# Clone the branch
!git clone -b {branch_name} {git_path}

# Path to data folder
data_dir = os.path.join(repo_name, "data", "explainable_ml_II")

# Verify files exist
print("Files in data folder:", os.listdir(data_dir))

# Load CSVs
zoo_path = os.path.join(data_dir, "zoo.csv")
class_path = os.path.join(data_dir, "class.csv")

zoo_df = pd.read_csv(zoo_path)
class_df = pd.read_csv(class_path)

print(zoo_df.head())
print(class_df.head())


Cloning into 'XAI'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 83 (delta 27), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (83/83), 1.22 MiB | 7.37 MiB/s, done.
Resolving deltas: 100% (27/27), done.
Files in data folder: ['zoo.csv', 'class.csv']

=== zoo.csv ===
  animal_name  hair  feathers  eggs  milk  airborne  aquatic  predator  \
0    aardvark     1         0     0     1         0        0         1   
1    antelope     1         0     0     1         0        0         0   
2        bass     0         0     1     0         0        1         1   
3        bear     1         0     0     1         0        0         1   
4        boar     1         0     0     1         0        0         1   

   toothed  backbone  breathes  venomous  fins  legs  tail  domestic  catsize  \
0        1         1         1         0     0     4     0         0        1   
1

### **PDP Plot**

*analysis of plots*

### **ICE Plot**

*analysis of plots*

### **ALE Plot**

*analysis of plots*

### **Analysis**

*discussion of findings*